In [1]:
# import neccesary libraries

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

import os


In [4]:
#Download cancer dataset directly from kaggle
# set the API keys and username
!pip install kaggle

os.environ['KAGGLE_USERNAME'] = "usefulness121"
os.environ['KAGGLE_KEY'] = "4f61b7e9ebe449d2e963fefbfa255b0f"

!kaggle datasets download -d uciml/breast-cancer-wisconsin-data # download dataset

usage: kaggle [-h] [-v]
              {competitions,c,datasets,d,kernels,k,models,m,files,f,config}
              ...
kaggle: error: unrecognized arguments: download dataset
